In [1]:
import pandas
import numpy as np

In [2]:
from sklearn.model_selection import GridSearchCV, train_test_split,cross_val_score,StratifiedKFold,KFold
from sklearn.metrics import confusion_matrix,accuracy_score,silhouette_score#,calinski_harabaz_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest,f_classif,SelectFdr
from sklearn import svm
from sklearn import preprocessing
from matplotlib import pyplot as plt
from sklearn.preprocessing import normalize,RobustScaler,StandardScaler,MinMaxScaler

In [3]:
# read data
methy = pandas.read_csv("D:/1_Jie_OIO/pancreatic_cancer/input data/methylation2.csv")
mrna = pandas.read_csv("D:/1_Jie_OIO/pancreatic_cancer/input data/mrna.csv")
mirna = pandas.read_csv("D:/1_Jie_OIO/pancreatic_cancer/input data/mirna.csv")
clinical_new = pandas.read_csv("D:/1_Jie_OIO/pancreatic_cancer/clinical_data3.csv")
# 
clinical_new = clinical_new.drop(['Unnamed: 0'], axis = 1)
clinical_new = clinical_new[['bcr_patient_barcode','vital_status','survival']]

methy = methy.drop(['Unnamed: 0'], axis=1)
mrna = mrna.drop(['Unnamed: 0'], axis=1)
mirna = mirna.drop(['Unnamed: 0'], axis=1)

methy = methy.set_index(['Group.1'])
mrna = mrna.set_index(['Group.1'])
mirna = mirna.set_index(['GeneSymbol'])

methy = methy.transpose()
mrna = mrna.transpose()
mirna = mirna.transpose()

# data log2 transformation
mrna = np.log2(mrna+1)
mirna = np.log2(mirna+1)

In [4]:
# vital status has to be 0/1 not 1/2
clinical_new[["vital_status"]] = clinical_new[["vital_status"]] -1

data_all = pandas.concat([methy,mrna,mirna],axis = 1)
data_all2 = data_all.loc[clinical_new['bcr_patient_barcode'],:]
data_all2

,A1BG,A1BG-AS1,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AACSP1,...,hsa-mir-937,hsa-mir-939,hsa-mir-940,hsa-mir-942,hsa-mir-944,hsa-mir-95,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b
TCGA.2J.AAB6,0.392125,0.443796,0.470523,0.587413,0.730783,0.481929,0.401216,0.081614,0.631308,0.789943,...,1.659146,0.310120,1.764731,0.970296,0.565228,2.806379,4.812427,5.570562,8.939383,14.976723
TCGA.2J.AAB8,0.503784,0.657865,0.671000,0.747297,0.745571,0.465296,0.555999,0.065306,0.688562,0.773821,...,0.835161,1.121734,1.500529,1.565665,0.606636,2.512059,3.122654,5.361682,8.882110,13.226844
TCGA.2J.AAB9,0.407150,0.551150,0.607378,0.714571,0.774303,0.478354,0.558286,0.096926,0.702072,0.846118,...,0.282327,0.282327,0.898881,1.448498,1.199706,2.290066,3.169561,5.349524,9.336851,14.191277
TCGA.2J.AABA,0.537019,0.692183,0.677414,0.646668,0.744123,0.485663,0.528758,0.062211,0.670515,0.819271,...,0.000000,0.253555,0.469137,1.724951,0.656654,0.656654,1.724951,4.576809,10.267290,13.527318
TCGA.2J.AABE,0.485248,0.632153,0.649158,0.756352,0.739516,0.485606,0.497274,0.086935,0.647609,0.786354,...,1.158019,0.611445,0.434945,1.553472,0.768687,3.266121,3.463099,5.252521,9.104412,13.860464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.XD.AAUL,0.430066,0.616884,0.601854,0.727783,0.716007,0.465204,0.560235,0.068362,0.656267,0.788165,...,1.521978,0.501787,1.168577,2.367276,1.168577,1.521978,3.587736,5.772403,9.206623,13.619919
TCGA.XN.A8T3,0.389799,0.508017,0.622184,0.761291,0.710781,0.480178,0.544202,0.065075,0.669153,0.834831,...,0.761530,0.231214,1.148449,2.824518,2.751920,4.279853,4.506384,5.888266,9.208170,14.478188
TCGA.YB.A89D,0.505117,0.648158,0.712331,0.791110,0.744692,0.486454,0.604904,0.065248,0.662704,0.822138,...,0.143124,0.143124,0.701117,1.765008,0.955049,2.427698,3.676404,5.942291,10.096909,13.867643
TCGA.YY.A8LH,0.568815,0.737544,0.500254,0.773945,0.655661,0.541389,0.582060,0.084887,0.619335,0.862767,...,1.051422,0.689655,1.340416,1.721752,0.205673,2.833695,5.227565,6.105980,9.825959,14.498457


In [5]:
# class label
label_all = pandas.read_csv("D:/1_Jie_OIO/pancreatic cancer/class_label.txt")
predictor_mrna = pandas.read_csv("D:/1_Jie_OIO/pancreatic cancer/predictor_mrna.txt")
predictor_mirna = pandas.read_csv("D:/1_Jie_OIO/pancreatic cancer/predictor_mirna.txt")
predictor_methy = pandas.read_csv("D:/1_Jie_OIO/pancreatic cancer/predictor_methy.txt")

In [6]:
mrna_train = mrna.loc[clinical_new['bcr_patient_barcode'],:]
mirna_train = mirna.loc[clinical_new['bcr_patient_barcode'],:]
methy_train = methy.loc[clinical_new['bcr_patient_barcode'],:]

define functions

In [22]:
# train the classifier for prediction
def train_svm(train,test):
    
    #normalizr
    train1 = MinMaxScaler().fit_transform(train.transpose()).transpose()
    test1 = MinMaxScaler().fit_transform(test.transpose()).transpose()
    train1 = RobustScaler().fit_transform(train1)
    test1 = RobustScaler().fit_transform(test1)
    
    train1 = pandas.DataFrame(train1,columns = train.columns)
    test1 = pandas.DataFrame(test1,columns = test.columns)
    
    train_select = SelectFdr(f_classif).fit(train1,label_all).get_support(indices = True)
    train2 = train1.iloc[:,train_select]
    #print(train2.columns)
    test2 = test1.iloc[:,train_select]
    
    svm_parameters = {
        #'kernel': ['rbf','sigmoid','poly','linear'],#
        'C': [0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.5],#,1,1.5,2,2.5,3,3.5,4,4.5,,5.5,6,6.5,7,7.5,8,8.5,9,9.5,10
        'coef0': [0.001,0.005,0.05,0.1,0.25,0.5,0.75,1],#,1.5,2,2.5
        #'degree' : [2,4,3],
        'gamma': [0.001,0.005,0.01,0.05,0.1,0.5,1,1.5,2,2.5,3]}#0.0001,0.0005,

    svm_tune = GridSearchCV(estimator=svm.SVC(kernel = 'sigmoid'),param_grid=svm_parameters,cv=3)#,iid = False,probability = True,class_weight = 'balanced'
    svm_tune.fit(train2,label_all)
    
    print(cross_val_score(svm_tune, train2,label_all, cv=3))
    print(svm_tune.best_params_)
    
    # predict
    pre = svm_tune.predict(test2)
    #pre2 = svm_tune.predict_proba(test2)
    
    return pre

ICGC mRNA-seq

In [11]:
au_rna = pandas.read_csv("D:/1_Jie_OIO/pancreatic_cancer/external dataset/PDAC_AU/rnaseq.tsv")
# au_rna = au_rna.drop(['Unnamed: 0'], axis=1)

au_rna = au_rna.set_index(['Unnamed: 0'])
# au_rna = au_rna.transpose()
# au_rna = np.log2(au_rna+1)
# au_rna

In [13]:
common_gene = predictor_mrna.intersection(au_rna.columns)
train_panel = mrna_train[list(common_gene)]
test_panel = au_rna[list(common_gene)]

In [14]:
common_gene = predictor_mrna.intersection(au_rna.columns)
train_panel = mrna_train[list(common_gene)]
test_panel = au_rna[list(common_gene)]
# predict
pred1 = train_svm(train_panel,test_panel)

ICGC mrna array

In [15]:
au_mrna_array = pandas.read_csv("D:/1_Jie_OIO/pancreatic_cancer/external dataset/PDAC_AU/mrna_array.tsv")

au_mrna_array = au_mrna_array.drop(['Unnamed: 0'],axis = 1)
au_mrna_array = au_mrna_array.set_index(["Group.1"])

au_mrna_array = au_mrna_array.transpose()

In [18]:
common_gene = predictor_mrna.intersection(au_mrna_array.columns)
train_panel = mrna_train[list(common_gene)]
test_panel = au_mrna_array[list(common_gene)]

pred2 = train_svm(train_panel,test_panel)

geo mrna array

In [20]:
geo_mrna = pandas.read_csv("D:/1_Jie_OIO/pancreatic_cancer/external dataset/PDAC_GEO_mrna/mrna.tsv")
geo_mrna = geo_mrna.drop(['Unnamed: 0'],axis = 1)
geo_mrna = geo_mrna.set_index(["Group.1"])

geo_mrna = np.log2(geo_mrna+1)
geo_mrna = geo_mrna.transpose()

In [23]:
common_gene = predictor_mrna.intersection(geo_mrna.columns)
train_panel = mrna_train[list(common_gene)]
test_panel = geo_mrna[list(common_gene)]

pred3 = train_svm(train_panel,test_panel)

geo pdac mirna

In [31]:
geo_mirna = pandas.read_csv("D:/1_Jie_OIO/pancreatic_cancer/external dataset/PDAC_GEO_mirna/mirna.tsv",sep = "\t")
geo_mirna = geo_mirna.set_index(['Unnamed: 0'])
# geo_mirna = geo_mirna.drop(['Unnamed: 0'],axis = 1)
geo_mirna = np.log2(geo_mirna+1)
# geo_mirna

In [32]:
common_gene = predictor_mirna.intersection(geo_mirna.columns)
train_panel = mirna_train[list(common_gene)]
test_panel = geo_mirna[list(common_gene)]

In [33]:
common_gene = predictor_mirna.intersection(geo_mirna.columns)
train_panel = mirna_train[list(common_gene)]
test_panel = geo_mirna[list(common_gene)]

pred4 = train_svm(train_panel,test_panel)

australia pdac methylation

In [34]:
au_methy = pandas.read_csv("D:/1_Jie_OIO/pancreatic_cancer/external dataset/PDAC_AU/methylation.tsv")

au_methy = au_methy.drop(['Unnamed: 0'], axis=1)
au_methy = au_methy.set_index(['Group.1'])
au_methy = au_methy.transpose()
# au_methy

In [35]:
common_gene = predictor_methy.intersection(au_methy.columns)
train_panel = methy_train[list(common_gene)]
test_panel = au_methy[list(common_gene)]

pred5 = train_svm(train_panel,test_panel)